In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 115.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
from gensim.models import FastText
import numpy as np
import torch
from transformers import BertModel, BertTokenizer

import pandas as pd
import numpy as np
import re, os, sys
from itertools import product

def read_nucleotide_sequences(file):
    if os.path.exists(file) == False:
        print('Error: file %s does not exist.' % file)
        sys.exit(1)
    with open(file) as f:
        records = f.read()
    if re.search('>', records) == None:
        print('Error: the input file %s seems not in FASTA format!' % file)
        sys.exit(1)
    records = records.split('>')[1:]
    fasta_sequences = []
    for fasta in records:
        array = fasta.split('\n')
        header, sequence = array[0].split()[0], re.sub('[^ACGTU-]', '-', ''.join(array[1:]).upper())
        header_array = header.split('|')
        name = header_array[0]
        label = header_array[1] if len(header_array) >= 2 else '0'
        label_train = header_array[2] if len(header_array) >= 3 else 'training'
        sequence = re.sub('U', 'T', sequence)
        fasta_sequences.append(sequence)
    return fasta_sequences

#!/usr/bin/env python
#_*_coding:utf-8_*_

import re

def check_fasta_with_equal_length(fastas):
    status = True
    lenList = set()
    for i in fastas:
        lenList.add(len(i[1]))
    if len(lenList) == 1:
        return True
    else:
        return False

def get_min_sequence_length(fastas):
    minLen = 10000
    for i in fastas:
        if minLen > len(i[1]):
            minLen = len(i[1])
    return minLen

def get_min_sequence_length_1(fastas):
    minLen = 10000
    for i in fastas:
        if minLen > len(re.sub('-', '', i[1])):
            minLen = len(re.sub('-', '', i[1]))
    return minLen
def readFasta(file):
    if os.path.exists(file) == False:
        print('Error: "' + file + '" does not exist.')
        sys.exit(1)

    with open(file) as f:
        records = f.read()

    if re.search('>', records) == None:
        print('The input file seems not in fasta format.')
        sys.exit(1)

    records = records.split('>')[1:]
    myFasta = []
    for fasta in records:
        array = fasta.split('\n')
        name, sequence = array[0].split()[0], re.sub('[^ARNDCQEGHILKMFPSTWYV-]', '-', ''.join(array[1:]).upper())
        myFasta.append([name, sequence])
    return myFasta


def extract_features(dna_sequences, vector_size=100, window=5, min_count=1):
    # Prepare data for FastText
    tokenized_sequences = [list(sequence) for sequence in dna_sequences]

    # Train FastText model
    model = FastText(tokenized_sequences, vector_size=vector_size, window=window, min_count=min_count)

    # Extract features
    features = np.zeros((len(dna_sequences), vector_size))
    for i, sequence in enumerate(tokenized_sequences):
        for token in sequence:
            features[i] += model.wv[token]
        features[i] /= len(sequence)

    return features

# Example usage
dna_sequences = ['ATCGATCGATCG', 'CGATCGATCGATCG']

fastas = read_nucleotide_sequences('/content/drive/MyDrive/Research Work/NLP-Features code/balanced_training_datasets/cd_ac4c_training_10.fasta')
features = extract_features(fastas)
data_csv=pd.DataFrame(features)
data_csv.to_csv('/content/drive/MyDrive/Research Work/NLP-Features code/balanced_training_datasets/FastText_feature_training_10.csv')

print(features)

In [1]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227141 sha256=5f48c0f3c19a934422880be353c3b9d24f5cd9ff0ae29aebca7bca249f706ee2
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [4]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.4 MB/s eta 0:00:00


In [3]:
import fasttext

# Path to your RNA sequence text file
data_path = "/content/drive/MyDrive/Research Work/NLP-Features code/Saweera_data/balanced_training_datasets/cd_ac4c_training_1.fasta"

# Train FastText model
model = fasttext.train_unsupervised(data_path, model='cbow', dim=100, epoch=10, minCount=1)

# Save the trained model
model.save_model("rna_fasttext_model.bin")

print("Model trained and saved successfully.")


Model trained and saved successfully.


In [8]:
import fasttext
from Bio import SeqIO
import numpy as np
import pandas as pd

# Load FastText model
# model = fasttext.load_model('/content/rna_fasttext_model.bin')  # Pre-trained FastText model, you can replace it with your own trained model
# Train FastText model
model = fasttext.train_unsupervised(input_file, model='cbow', dim=100, epoch=10, minCount=1)

# Function to convert RNA sequence into numerical features
def sequence_to_features(sequence, model):
    # Tokenize RNA sequence into k-mers (n-grams)
    k = 3  # You can adjust the k-mer size
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

    # Compute FastText embeddings for each k-mer
    embeddings = [model.get_word_vector(kmer) for kmer in kmers]

    # Calculate mean embedding as the feature representation
    if embeddings:
        feature = np.mean(embeddings, axis=0)
    else:
        feature = np.zeros(model.get_dimension())  # Return zero vector if no k-mers found

    return feature

# Function to read RNA sequences from FASTA file and convert to features
def fasta_to_features(input_file, model):
    sequences = []
    features = []

    for record in SeqIO.parse(input_file, "fasta"):
        sequences.append(str(record.seq))
        features.append(sequence_to_features(str(record.seq), model))

    return sequences, np.array(features)

# Read RNA sequences from FASTA file and convert to features
input_file = "/content/drive/MyDrive/Research Work/NLP-Features code/Saweera_data/balanced_training_datasets/cd_ac4c_training_1.fasta"
sequences, features = fasta_to_features(input_file, model)

# Create DataFrame for features
columns = [f"feature_{i+1}" for i in range(features.shape[1])]
df = pd.DataFrame(features, columns=columns)
df.insert(0, "Sequence", sequences)

# Save features as CSV
output_file = "rna_features.csv"
df.to_csv(output_file, index=False)

print("Features saved to", output_file)


Features saved to rna_features.csv
